#Creating a Streamlit application to validate entities

A Streamlit app has been created to validate the NER dataset generated by the LLM. You can accept or remove labeled phrases from the dataset. The labeled words will be highlighted.




## Imports and environment setup

In [ ]:
!pip install streamlit st-annotated-text pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
import os
from pyngrok import conf
import pandas as pd

In [ ]:
from google.colab import userdata
# Salva la configurazione di ngrok
conf.get_default().auth_token = userdata.get("NGROK_AUTHTOKEN")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Constants

In [ ]:
DATASET="/content/drive/MyDrive/SanRaffaele/Data/Dataset NER/NER_LLAMA70B.csv"

## Streamlit app

In [ ]:
os.environ["STREAMLIT_DISABLE_WATCHDOG_WARNINGS"] = "true"


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import ast

st.set_page_config(page_title="NER Validator", layout="wide")
st.title("📌 NER Dataset Validator")

if "data" not in st.session_state:
  uploaded = st.file_uploader("Carica CSV", type="csv")
  if uploaded:
      df = pd.read_csv(uploaded)
      df["label"] = df["label"].apply(ast.literal_eval)
      st.session_state.data = df
      st.session_state.idx = 0
      st.session_state.results = []
      st.session_state.correct = 0
      st.session_state.wrong = 0
  else:
      st.stop()  # Fermiamo l'app finché non c'è un file

df = st.session_state.data

total = len(df)
st.sidebar.metric("Totale righe", total)
st.sidebar.metric("Corrette", st.session_state.correct)
st.sidebar.metric("Sbagliate", st.session_state.wrong)
st.sidebar.progress(st.session_state.idx / total if total > 0 else 0)

def render_annotated(row):
  tokens = row["frase"].split()
  labels = row["label"]
  from annotated_text import annotated_text as show
  annotated = []
  ent = None
  for token, lab in zip(tokens, labels):
      if lab.startswith("B-"):
          ent = lab[2:]
          annotated.append((token, ent))
      elif lab.startswith("I-") and ent:
          annotated.append((token, ent))
      else:
          ent = None
          annotated.append(token+' ')
  show(*annotated)

if st.session_state.idx < total:
  row = df.iloc[st.session_state.idx]
  st.subheader(f"Riga {st.session_state.idx+1} / {total}")
  render_annotated(row)

  col1, col2 = st.columns(2)
  if col1.button("✅ Corretta"):
      st.session_state.results.append({**row, "correct": True})
      st.session_state.correct += 1
      st.session_state.idx += 1
      st.rerun()
  if col2.button("❌ Sbagliata"):
      st.session_state.results.append({**row, "correct": False})
      st.session_state.wrong += 1
      st.session_state.idx += 1
      st.rerun()
else:
  st.success("✅ Validazione completata!")
  df_res = pd.DataFrame(st.session_state.results)
  tp = len(df_res[df_res["correct"]==True])
  tn = len(df_res[df_res["correct"]==False])
  fp = 0
  fn = 0

  acc = (tp + tn) / total
  prec = tp/(tp+fp) if tp+fp>0 else 0
  rec = tp/(tp+fn) if tp+fn>0 else 0
  f1 = 2*prec*rec/(prec+rec) if prec+rec>0 else 0

  col1, col2, col3, col4 = st.columns(4)
  col1.metric("Accuracy", f"{acc:.2f}")
  col2.metric("Precision", f"{prec:.2f}")
  col3.metric("Recall", f"{rec:.2f}")
  col4.metric("F1 score", f"{f1:.2f}")

  st.download_button("Scarica validato", data=df_res.to_csv(index=False), file_name="validated.csv")


Writing app.py


Each user has a user_id obtained from the URL or generated as a UUID.

The state.json file stores row assignments and validation results for all users.

Each user sees a unique row that has not yet been validated or assigned.

Once validated, the row is no longer shown to anyone.

The results are displayed only when all rows have been validated.

In [ ]:
  %%writefile app.py

  import streamlit as st
  import pandas as pd
  import ast
  import json
  import os
  import uuid
  from annotated_text import annotated_text as show

  DATA_FILE = "/content/drive/MyDrive/SanRaffaele/Data/Dataset NER/NER_LLAMA70B.csv"
  STATE_FILE = "/content/drive/MyDrive/SanRaffaele/Data/Dataset NER/state.json"


  def load_state():
      if os.path.exists(STATE_FILE):
          with open(STATE_FILE, "r") as f:
              return json.load(f)
      else:
          return {"assigned": {}, "results": {}}

  def save_state(state):
      with open(STATE_FILE, "w") as f:
          json.dump(state, f)

  def get_next_row(df, state, user_id):
      # Se l'utente ha già una riga assegnata non validata, torna quella
      if user_id in state["assigned"]:
          idx = state["assigned"][user_id]
          if str(idx) not in state["results"]:
              return idx
          else:
              del state["assigned"][user_id]

      assigned_indices = set(state["assigned"].values())
      validated_indices = set(int(i) for i in state["results"].keys())

      for i in range(len(df)):
          if i not in assigned_indices and i not in validated_indices:
              state["assigned"][user_id] = i
              save_state(state)
              return i
      return None

  def render_annotated(row):
      tokens = row["frase"].split()
      labels = row["label"]
      annotated = []
      ent = None
      for token, lab in zip(tokens, labels):
          if lab.startswith("B-"):
              ent = lab[2:]
              annotated.append((token, ent))
          elif lab.startswith("I-") and ent:
              annotated.append((token, ent))
          else:
              ent = None
              annotated.append(token+' ')
      show(*annotated)

  # --- Streamlit app setup ---

  st.set_page_config(page_title="NER Validator ", layout="wide")
  st.title("📌 NER Dataset Validator MULTI ")

  # Inizializza session_id se non esiste
  if "session_id" not in st.session_state:
      st.session_state.session_id = str(uuid.uuid4())

  # Ottieni user_id da query params o genera fallback
  params = st.query_params
  user_id = params.get("user", [f"user_{st.session_state.session_id}"])[0]
  st.session_state.user_id = user_id

  st.write(f"👤 User ID: {user_id}")

  # Caricamento dati
  if "data" not in st.session_state:

        df = pd.read_csv(DATA_FILE)
        # Converti stringhe label in liste reali
        df["label"] = df["label"].apply(ast.literal_eval)
        st.session_state.data = df

  df = st.session_state.data

  # Carica stato globale
  state = load_state()

  idx = get_next_row(df, state, user_id)

  if idx is None:
      st.success("✅ Tutte le righe sono state validate!")

      results = state["results"]
      if results:
          rows = []
          for k,v in results.items():
              row = df.iloc[int(k)].to_dict()
              row["correct"] = v["correct"]
              rows.append(row)
          df_res = pd.DataFrame(rows)

          tp = len(df_res[df_res["correct"]==True])
          tn = len(df_res[df_res["correct"]==False])
          fp = 0
          fn = 0

          total = len(df)
          acc = (tp + tn) / total
          prec = tp/(tp+fp) if tp+fp>0 else 0
          rec = tp/(tp+fn) if tp+fn>0 else 0
          f1 = 2*prec*rec/(prec+rec) if prec+rec>0 else 0

          col1, col2, col3, col4 = st.columns(4)
          col1.metric("Accuracy", f"{acc:.2f}")
          col2.metric("Precision", f"{prec:.2f}")
          col3.metric("Recall", f"{rec:.2f}")
          col4.metric("F1 score", f"{f1:.2f}")

          st.download_button("Scarica validato", data=df_res.to_csv(index=False), file_name="validated.csv")
  else:
      row = df.iloc[idx]
      st.subheader(f"Riga {idx+1} / {len(df)}")

      render_annotated(row)

  col1, col2 = st.columns(2)
  if col1.button("✅ Corretta"):
      state["results"][str(idx)] = {"correct": True}
      if user_id in state["assigned"]:
          del state["assigned"][user_id]
      save_state(state)
      st.rerun()

  if col2.button("❌ Sbagliata"):
      state["results"][str(idx)] = {"correct": False}
      if user_id in state["assigned"]:
          del state["assigned"][user_id]
      save_state(state)
      st.rerun()


Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Kill eventuali vecchi tunnel
ngrok.kill()

# Crea il tunnel
public_url = ngrok.connect(8502)
print("🌍 URL pubblico:", public_url)
# Disattiva file watcher (evita crash su Colab)

# Avvia Streamlit
!STREAMLIT_SERVER_FILE_WATCHER_TYPE=none streamlit run app.py --server.port=8502 &> /content/logs.txt &


🌍 URL pubblico: NgrokTunnel: "https://fead04bc2db8.ngrok-free.app" -> "http://localhost:8502"
